### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
from datetime import datetime 

# Read Ironore_price_market_index_test.csv File and store into Pandas data frame
commodity_to_load = "Resources/Ironore_price_market_index_test.csv"
purchase_data = pd.read_csv(commodity_to_load)
#purchase_data

In [2]:
#remove empty columns, fix naming error and bring Date in line with other dataframes
clean_data = purchase_data[['Date','Iron Price','Silver Price','Gold Price',' Copper Price','S&P GSCI Price']]
clean_data = clean_data.rename(columns={' Copper Price': 'Copper_Price',
                                       'Iron Price':'Iron_Price',
                                        'Silver Price':'Silver_Price',
                                        'Gold Price':'Gold_Price',
                                       'S&P GSCI Price':'S&P_GSCI_Price'})

clean_data['Date'] = clean_data['Date'].str[2:]
#clean_data

#save data to Commodities_clean.csv
clean_data.to_csv('Outputs/Commodities_clean.csv')

In [3]:
iron_data = clean_data[['Date','Iron_Price']]


In [4]:
#https://www.kaggle.com/docs/api
#https://stackoverflow.com/questions/64393520/r-not-working-as-lineterminator-within-python-csv-writer
# Read all_perth_310121.csv File and store into Pandas data frame
perth_to_load = "Resources/all_perth_310121.csv"

perth_data = pd.read_csv(perth_to_load)


In [5]:
#fix up Date_sold format errors and bring in line with other datasets
perth_data["DATE_SOLD"] = perth_data["DATE_SOLD"].str.replace('\r','')
perth_data["DATE_SOLD"] = perth_data["DATE_SOLD"].str.replace('-','/')
#perth_data

In [6]:
#Limit dataframe to variables of interest
clean_perth = perth_data[['SUBURB',
                          'PRICE',
                          'BEDROOMS',
                          'BATHROOMS',
                          'DATE_SOLD',
                          'POSTCODE']]
clean_perth = clean_perth.rename(columns={'DATE_SOLD': 'Date'})
#Add unique ID and add City for later merging with Sydney data
clean_perth.insert(0, 'ID', range(1, 1 + len(clean_perth)))
clean_perth['CITY'] = "Perth"
#clean_perth

In [7]:
#merge Perth data with Commodities data 
Perth_commodities_df = pd.merge(clean_data, clean_perth, on="Date",how="inner")
#Perth_commodities_df

In [8]:
#save data to perth_clean.csv
Perth_commodities_df.to_csv('Outputs/perth_housing.csv')

In [9]:
#Create Perth data frame with Iron ore prices
perth_iron_df = pd.merge(iron_data, clean_perth, on="Date",how="inner")
perth_iron_df.to_csv('Outputs/perth_iron_housing.csv')


In [10]:
# Read SydneyHousePrices.csv File and store into Pandas data frame
Sydney_to_load = "Resources/SydneyHousePrices.csv"
Sydney_data = pd.read_csv(Sydney_to_load)

In [11]:
#Add City indicator for when merged with Perth data
Sydney_data["CITY"] = "Sydney"
#sanity check to see earliest date
#Sydney_data.sort_values(by=['Date'])


In [12]:
#Change the Date from string to datetime then convert to desired format 
Sydney_data['Date'] = pd.to_datetime(Sydney_data['Date'], errors='coerce')
Sydney_data['Date'] = Sydney_data["Date"].dt.strftime("%m/20%y")
#Sydney_data

In [13]:
#Rename columns matching Perth data to combine later
clean_Sydney = Sydney_data.rename(columns={'Id':'ID',
                          'suburb':'SUBURB',
                          'sellPrice':'PRICE',
                          'bed':'BEDROOMS',
                          'bath':'BATHROOMS',
                          'postalCode':'POSTCODE'})
#clean_Sydney

In [14]:
#merge commodity prices with Sydney data
sydney_commodities_df = pd.merge(clean_data, clean_Sydney, on="Date",how="inner")
#sydney_commodities_df
#save data to sydney_clean.csv
sydney_commodities_df.to_csv('Outputs/sydney_housing.csv')

In [15]:
#Create Sydney data frame with Iron ore prices
sydney_iron_df = pd.merge(iron_data, clean_Sydney, on="Date",how="inner")
sydney_iron_df.to_csv('Outputs/sydney_iron_housing.csv')
sydney_iron_df

,Date,Iron_Price,ID,SUBURB,POSTCODE,PRICE,BEDROOMS,BATHROOMS,car,propType,CITY
0,12/2000,12.45,92373,Bondi Beach,2026,3565000,10.0,5,3.0,house,Sydney
1,02/2001,12.99,147101,Padstow,2211,960000,4.0,2,2.0,house,Sydney
2,03/2001,12.99,20554,Allambie Heights,2100,896000,4.0,2,2.0,house,Sydney
3,04/2001,12.99,75403,Cammeray,2062,463000,2.0,1,2.0,townhouse,Sydney
4,09/2001,12.99,32744,Cremorne,2090,372000,2.0,1,1.0,house,Sydney
...,...,...,...,...,...,...,...,...,...,...,...
199499,07/2019,115.55,197646,Ruse,2560,692000,6.0,2,2.0,house,Sydney
199500,07/2019,115.55,197647,Ambarvale,2560,549950,3.0,1,2.0,house,Sydney
199501,07/2019,115.55,197648,St Helens Park,2560,539000,3.0,1,1.0,house,Sydney
199502,07/2019,115.55,198931,Illawong,2234,1770000,4.0,2,6.0,house,Sydney


In [16]:
#combine Perth and Sydney data into one DataFrame
complete_city_df = pd.concat([clean_Sydney, clean_perth], ignore_index=True)
#limit variables to match eachother
complete_city_df = complete_city_df[[
                          'SUBURB',
                          'PRICE',
                          'BEDROOMS',
                          'BATHROOMS',
                          'Date',
                          'POSTCODE',
                           'CITY']]

#merge commodity prices with city data and add unique ID for each sale
complete_df = pd.merge(clean_data, complete_city_df, on="Date",how="inner")
complete_df.insert(0, 'ID', range(1, 1 + len(complete_df)))
complete_df

,ID,Date,Iron_Price,Silver_Price,Gold_Price,Copper_Price,S&P_GSCI_Price,SUBURB,PRICE,BEDROOMS,BATHROOMS,POSTCODE,CITY
0,1,01/2000,12.45,5.26,283.30,5.26,"2,970.29",Mahogany Creek,98000,2.0,1,6072,Perth
1,2,01/2000,12.45,5.26,283.30,5.26,"2,970.29",Mardella,235000,4.0,1,6125,Perth
2,3,02/2000,12.45,5.10,293.65,5.10,"3,155.86",Upper Swan,706000,3.0,2,6069,Perth
3,4,03/2000,12.45,4.94,276.75,4.94,"3,119.23",Bickley,165000,3.0,1,6076,Perth
4,5,03/2000,12.45,4.94,276.75,4.94,"3,119.23",Mariginiup,196000,4.0,2,6078,Perth
...,...,...,...,...,...,...,...,...,...,...,...,...,...
232941,232942,12/2020,158.15,26.49,"1,891.10",26.49,"1,976.96",Bassendean,550000,3.0,1,6054,Perth
232942,232943,12/2020,158.15,26.49,"1,891.10",26.49,"1,976.96",Palmyra,629000,3.0,1,6157,Perth
232943,232944,12/2020,158.15,26.49,"1,891.10",26.49,"1,976.96",High Wycombe,480000,4.0,2,6057,Perth
232944,232945,12/2020,158.15,26.49,"1,891.10",26.49,"1,976.96",Secret Harbour,754000,4.0,3,6173,Perth


In [17]:
#save data to complete_house_commodities.csv
complete_df.to_csv('Outputs/complete_house_commodities.csv')

In [18]:
iron_city_df = pd.concat([perth_iron_df, sydney_iron_df], ignore_index=True)
#limit variables to match eachother
iron_city_df = iron_city_df[['Date',
                             'Iron_Price',
                             'SUBURB',
                             'PRICE',
                             'BEDROOMS',
                             'BATHROOMS',
                             'POSTCODE',
                             'CITY']]
iron_city_df['Date'] = pd.to_datetime(iron_city_df['Date'], errors='coerce').dt.to_period('M')
iron_city_df = iron_city_df[(iron_city_df['Date'] >= '2007-01')
                           & (iron_city_df['Date'] <= '2019-06')]
#iron_city_df
iron_city_df.to_csv('Outputs/complete_house_iron.csv')
iron_city_df

,Date,Iron_Price,SUBURB,PRICE,BEDROOMS,BATHROOMS,POSTCODE,CITY
581,2007-01,36.63,Menora,875000,2.0,1,6050,Perth
582,2007-01,36.63,Menora,720000,3.0,1,6050,Perth
583,2007-01,36.63,Peppermint Grove,2100000,5.0,3,6011,Perth
584,2007-01,36.63,Walliston,627000,4.0,3,6076,Perth
585,2007-01,36.63,Karawara,860000,4.0,2,6152,Perth
...,...,...,...,...,...,...,...,...
232703,2019-06,112.90,Bangor,775000,3.0,1,2234,Sydney
232704,2019-06,112.90,Menai,740000,3.0,1,2234,Sydney
232705,2019-06,112.90,Illawong,1345000,4.0,3,2234,Sydney
232706,2019-06,112.90,Barden Ridge,865000,3.0,2,2234,Sydney


In [19]:
#create data drame of 3 bedroom houses
three_bed_df = iron_city_df[iron_city_df['BEDROOMS']==3]
three_bed_df.to_csv('Outputs/three_bed_housing.csv')

In [20]:
#create data drame of 4 bedroom houses
four_bed_df = iron_city_df[iron_city_df['BEDROOMS']==4]
four_bed_df.to_csv('Outputs/four_bed_housing.csv')

In [21]:
#create data drame of 4 bedroom 2 bathroom houses
four_bed_2bath_df = four_bed_df[four_bed_df['BATHROOMS']==2]
four_bed_2bath_df.to_csv('Outputs/four_bed_two_bath_housing.csv')